# 3. Kunstige neurale netværk

Kunstige neurale netværk (ANNs) giver en alsidig måde at opbygge enhver maskinlæringsmodel på. Ideen er inspireret af strukturen og funktionen af den menneskelige hjerne. Neurale netværk har vist sig at være kraftfulde til at løse komplekse problemer med klassifikation og andre opgaver. I denne klasse vil vi forklare grundlæggende om neurale netværk, deres komponenter, træningsprocessen og validering af ANNs.

Der findes flere biblioteker til Python, der implementerer ANN. De mest kendte er [Tensorflow](https://www.tensorflow.org) og [PyTorch](https://pytorch.org) (der findes også et bibliotek, som tager det bedste fra begge, [Keras](https://keras.io)). Til eksemplerne i denne klasse vil vi bruge PyTorch, som i disse dage sandsynligvis er det mest udbredte og vel dokumenterede.

Installer biblioteket ved at køre følgende kode (efter det, tilføj `#` foran denne linje for at undgå at køre den igen):

In [ ]:
! pip install torch torchinfo

Som du måske har lagt mærke til, har vi installeret to biblioteker her, PyTorch (`torch`) og et supplement, der vil hjælpe os med at opdage strukturen af ANNs (`torchinfo`).

Inden vi fortsætter, lad os splitte vores data op i henholdsvis træningssæt og testsæt igen, som vi gjorde det i den forrige klasse. Vi vil senere bruge træningssættet til læreprocessen og testsættet til at vurdere, hvor god den trænede model er.

Rent teknisk har vi brug for tre sæt i dette tilfælde:

- *træningssæt*, som bruges til læring
- *valideringssæt*, som bruges til at optimere modellen (finde den bedste under læring)
- *testsæt*, som bruges til at vurdere kvaliteten af den endelige model

For at forenkle eksemplerne vil vi bruge *testsættet* til både validering og test, men når du arbejder med reelle tilfælde, så sørg for at de er adskilte, og du har alle tre sæt (vi vil bruge denne tilgang i den næste klasse).

Som i den forrige klasse tager vi simpelthen hver femte måling som testsæt, og senere vil vi lære, hvordan man gør det på en bedre måde.

In [ ]:
# load data from CSV file as data frame
import pandas as pd
d = pd.read_csv("Iris.csv")

# generate logical values for train and test set measurements (rows of data frame)
train_ind = d["Id"] % 5 != 0
test_ind = d["Id"] % 5 == 0

# make the split
d_train = d.loc[train_ind]
d_test = d.loc[test_ind]

# show size of each set
(d_train.shape, d_test.shape)

Hvad er et neuralt netværk?

En neural netværk er simpelthen et sæt noder, kendt som *neuroner*, der er forbundet med hinanden. I den simpleste tilfælde består et neuralt netværk kun af en node som vist på billedet nedenfor.

<img src="./illustrations/Neuron.png" style="width:500px; height:400px;"/>

Hver neuron har en række inputs (vist som $X_1$, $X_2$, $X_3$ og $X_4$ på venstre side af billedet) og outputtet (vist som $\hat{Y}$ på højre side). Den udfører en meget simpel opgave - den tager alle tal, som den modtager fra inputtet, og anvender en matematisk funktion, som beregner en outputværdi baseret på inputtet.

I den simpleste tilfælde beregner den en vægtet sum (kendt som *linearkombination*) af disse tal og sender det beregnede tal til outputtet. Matematisk kan vi skrive dette som følger:

$\hat{Y} = (X_{1}\times W_{1}) + (X_{2}\times W_{2}) + (X_{3}\times W_{3}) + (X_{4}\times W_{4}) + Bias$

Værdierne $W_1$,...,$W_4$ er *vægte*, som hvert input bidrager til outputtet med.

Forestil dig, at inputtene er målingerne fra den første række af Iris-datasættet: 

$X = [5.1,3.5,1.4,0.2]$ 

og vægtene er: 

$W = [0.1, 0.2, 2.0, 5.0]$. 

Lad os antage, at bias er $1.0$. Så vil outputværdien for vores neuron være:

$\hat{Y} = 5.1 \times 0.1 + 3.5 \times 0.2 + 1.4 \times 2.0 + 0.2 \times 5.0 + 1.0 = 6.01$

Så simpelt er det.

Her er hvordan vi kan implementere dette ene neuron-baserede kunstige neurale netværk i Python ved hjælp af PyTorch biblioteket:

In [ ]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    """ class for one-neuron ANN model """

    def __init__(self):
        # initialize the parent (super) class for ANN model
        super(SimpleModel, self).__init__()
        # define all layers with neurons and their properties
        self.layer1 = nn.Linear(4, 1)

    def forward(self, x):
        """ takes vector with input values 'x', computes and returns the output """
        y_hat = self.layer1(x)
        return y_hat

Som du kan se, koden er lidt mere kompleks sammenlignet med det, vi har haft før. I denne kode opretter vi en *klasse* `SimpleModel` oven på en anden klasse `nn.Module`, som allerede er implementeret i PyTorch.

>Du kan tænke på en klasse som en opskrift, en byggevejledning, et sæt instruktioner. Forestil dig, at du vil bygge et hus. Der er et sæt grundlæggende instruktioner, der fortæller, hvordan man bygger fundamentet til huset, lægger afløbsrør osv. Derudover har det nogle typiske instruktioner, f.eks. hvordan man bygger en mur af mursten. Der er ingen grund til at opfinde den dybe tallerken, så når du vil lave instruktioner til at bygge et helt hus (et typisk projekt), med vægge, tag osv., kan du tage det grundlæggende sæt som basis og udvide det med din egen del.

Samme idé her, `nn.Model` er et grundlæggende sæt instruktioner om, hvordan man laver et neuralt netværk i PyTorch, og det inkluderer en masse ting, du ikke behøver at bekymre dig om. Du tager det simpelthen som basis og udvider det med dine egne specifikke dele - hvilke noder du vil bruge, hvor mange indgange de har, hvor mange udgange osv. Så denne nye instruktion er klassen `SimpleModel`.

Som du kan se, har vi tilføjet to metoder til klassen.

Den første metode, `__init__`, er nødvendig for at initialisere din model. I begyndelsen indeholder den:

```python
super(SimpleModel, self).__init__()
```

hvilket fortæller Python at foretage alle forberedelser baseret på det grundlæggende sæt instruktioner (f.eks. lave husets fundament).

Og derefter definerer du dit netværk - hvor mange neuroner, hvilken type neuroner og definerer deres egenskaber. I dette tilfælde har vi en lineær neuron med 4 indgange og 1 udgang - præcis hvad vi brugte i eksemplet ovenfor.

**Du kan tænke på `__init__()` som en metode, der bygger huset ved hjælp af dine instruktioner.** Ikke malet, uden møbler, men et helt, fuldt fungerende hus.

Den anden metode i denne klasse, `forward()`, bruges hver gang du vil anvende din model på givne indgange for at producere outputtet. Metoden "forbinder" neuronerne, den sikrer, at indgangene går gennem neuronerne i korrekt rækkefølge og returnerer det resulterende output til sidst.

**Du kan tænke på `forward()` som en metode, der giver dig mulighed for at bruge det hus, du har skabt.** Den bruges altid i dens nuværende tilstand. Så hvis du anvender metoden `forward()` på et nyoprettet hus, vil det ikke fungere godt, da dit hus har bare vægge og ingen møbler. Men du kan selvfølgelig bo der.

Her er et eksempel på, hvordan vi kan bruge modellen:

In [ ]:
# initialize the model
model = SimpleModel()

# define values for input
X = torch.tensor([[5.1, 3.5, 1.4, 0.2]])

# send input to the model and get the output
y_hat = model(X)
y_hat

Man kan se, at hver gang vi skal give nogle tal til PyTorch, er det ikke nok bare at kombinere dem til en 1D-liste ved at bruge firkantede parenteser, f.eks.:

```python
X = [5.1, 3.5, 1.4, 0.2] 
```

eller lave en 2D-liste som:

```python
X = [[5.1, 3.5, 1.4, 0.2]]
```

Vi skal også konvertere listen til en speciel type, `torch.tensor`. Dette ligner det, vi gjorde i den første klasse, for at oprette NumPy-arrays, vi gav værdier som en liste og brugte derefter en speciel metode, der fortæller Python - lav det til et NumPy array:

```python
X = np.array([[5.1, 3.5, 1.4, 0.2]])
```

Visuelt set er *Tensor* det samme som et array, det kan være 1D (vektor), 2D (matrix), 3D osv. Så Torch tensor ligner NumPy array, som du allerede kender. Men i virkeligheden er der en forskel. For Python er de to forskellige dataobjekter fra to forskellige biblioteker, og du kan anvende forskellige metoder og operatører på hver af dem. Det er derfor vigtigt at "fortælle" Python, at dette ikke bare er en liste eller NumPy array, men en PyTorch tensor.

>Et af eksemplerne her kan være biler og servicecentre. Hvis du har en Ford, vil du ikke gå til Hyundais servicecenter for f.eks. at skifte bremser og motorolie. Fordi deres mekanikere ikke er certificeret til at arbejde med dette mærke. Selvom begge er biler, ser de lignende ud, fungerer lignende, og du kan nemt køre begge, er de ikke identiske.  

En af grundene til at bruge Torch tensorer i stedet for NumPy arrays er, at tensorerne er designet til at arbejde med [GPU](https://en.wikipedia.org/wiki/Graphics_processing_unit) - de kraftfulde grafikkort, som vi f.eks. har i spil computere. Derudover fungerer det kun med specifikke GPU'er, fremstillet af virksomheden [NVIDIA](https://www.nvidia.com/da-dk/geforce/graphics-cards/). Hvis du er gamer, har du sandsynligvis hørt om grafikkort som GTX 3090. Da neurale netværk kræver en masse computerkraft, gør brugen af GPU dem meget hurtigere, og PyTorch er designet til at arbejde på en GPU først. Hvis du ikke har en GPU, kan den også arbejde på en konventionel processor, men beregningen vil være langsom. Især hvis du har store datasæt med tusindvis af objekter.

Så selvom det er lidt irriterende, at vi skal tilføje `torch.tensor()`, vil du senere finde ud af, at det ikke er et stort problem.

Hvis vi vender tilbage til vores kode og dens beregnede værdi, kan du se, at outputtet ikke er $6.01$, desuden hver gang du kører denne kode (prøv at klikke på *Kør*-knappen flere gange), vil du få en ny output. Fordi når du initialiserer ANN-modellen, bruger den tilfældige tal for dine vægte.

Du kan ændre dette og tildele vægtene manuelt efter initialisering:

In [ ]:
# define weights and bias
W = torch.tensor([[0.1, 0.2, 2.0, 5.0]])
Bias = torch.tensor([1.0])

# set weights and bias manually for the neurons on layer "layer1"
model.layer1.weight = nn.Parameter(W)
model.layer1.bias = nn.Parameter(Bias)

# forward pass through the model
y_hat = model(X)
y_hat

Nu er det $6,01$! Du kan bemærke, at værdien, vi får som output, også er en tensor.

Desuden med Torch kan du også beregne lineær kombination manuelt:

In [ ]:
(X * W).sum() + Bias

 Lad os nu implementere klassifikationsreglen for *Virginica*-prøverne, som vi oprettede i en tidligere klasse. Hvis du husker, sammenlignede vi kronbladets bredde ($X_4$ i vores tilfælde) af en blomst med 1,7, og hvis værdien var over denne tærskel, klassificerede vi denne blomst som *Virginica*.

Her er de vægte og bias, der kan implementere denne regel:

In [ ]:
# define weights and bias
W = torch.tensor([[0.0, 0.0, 0.0, 1.0]])
Bias = torch.tensor([-1.7])

# set weights and bias manually
model.layer1.weight = nn.Parameter(W)
model.layer1.bias = nn.Parameter(Bias)

# forward pass through the model
y_hat = model(X)

# show result and apply threshold
(y_hat, y_hat > 0)

Som du kan se, alle vægte bortset fra den sidste er sat til nul, mens vægten for $X_4$ er lig med én. Dette giver outputtet, $\hat{Y}$, simpelthen lig med værdien for $X_4$, som i vores tilfælde er Kronbladets bredde. Derefter anvender vi bias, så vi trækker vores tærskel fra. Dette giver følgende:

$\hat{Y} = X_4 - 1.7$

Tilsyneladende, hvis *Kronbladets bredde* er over tærsklen, vil outputværdien være negativ, og når den er under, vil værdien være positiv. Så vi kan træffe en klassifikationsbeslutning ved simpelthen at sammenligne den med 0, som vi gjorde ovenfor.

Nu lader vi anvende dette netværk med manuelt indstillede vægte på testsettet:

In [ ]:
# take only columns with measurements and convert them to Torch tensor
X_test_values = d_test.iloc[:, 1:5].values
X_test = torch.tensor(X_test_values).float()

# apply the model
y_hat = model(X_test)

# show the results
y_hat > 0

Ja, vi fik mange negative tal i begyndelsen, hvor vi har blomster af *Setosa* og *Versicolor* arterne, og positive tal i slutningen, hvor vi har blomster af målklassen, *virginica*.

Lad os kombinere outputtet og referenceværdierne til et datasæt for bedre synlighed:

In [ ]:
# convert output to numpy array (transpose -> detach from model -> convert to NumPy array)
y_hat_arr = y_hat.t().detach().numpy()
y_hat_arr

In [ ]:

# combine with reference values
res = pd.DataFrame({
    "Reference": d_test["Species"],
    "Predicted": y_hat_arr[0],
    "Is virginica": y_hat_arr[0] > 0
})
res

Vi fik helt præcis de samme resultater som i den sidste klasse!

Men hvordan lader vi modellen finde klassificeringsbeslutningen automatisk, baseret på de givne data? Vi skal træne modellen! Men hvad skal vi bruge som træningskriterie?

## Tabelfunktion

Hele ideen med træning af et kunstigt neuralt netværk (ANN) er at finde vægtene (og forskydningerne og andre parametre, hvis der er nogen) for alle neuroner, som vil gøre udgangen så tæt på den ønskede som muligt.

Men hvordan måler vi afstanden mellem modellens output og det ønskede output? Dette er hvad der defineres som en *tabe*. Taben er et tal, en statistik, som fortæller, hvor stor forskellen er mellem det ønskede output, $Y$, og det forudsagte output, $\hat{Y}$.

Som du husker, ønsker vi at lave klassifikation for *Virginica* blomster. Lad os definere det ideelle output til at være 1 for *virginica* og -1 for de andre. Sådan opretter du det:

In [ ]:
# get the classes labels for the training set
c = d_test["Species"]

# compare the labels with target class then multiply the result to 2 and subtract 1
# if the result is False, it will be treated as 0: 0 * 2 - 1 = -1
# if the result is True, it will be treated as 1: 1 * 2 - 1 = 1
y_dummy = (c == "virginica") * 2 - 1
y_dummy

In [ ]:
# convert the values from the data frame column to torch array
y = torch.tensor([y_dummy.values])
y

Det eneste problem, vi har, er, at værdierne præsenteres som rækker, og vi har brug for dem som en kolonne. Lad os tilføje transponering og også konvertere dem fra heltal til flydende punkt numre:

In [ ]:
# convert the values to torch array with 1 column and 120 rows
y = torch.tensor([y_dummy.values]).float().t()
y

Hver gang vi får output fra modellen, skal vi sammenligne den med disse reference y-værdier og beregne en statistik, der fortæller, hvor stor forskellen er — tabet. Den enkleste statistik, der vil gøre det for dig, kaldes middelkvadreret fejl (MSE). I dette tilfælde tager du simpelthen en forskel mellem de to vektorværdier, kvadrerer denne forskel og beregner gennemsnittet (middelværdien).

Her er hvordan man implementerer det:

In [ ]:
def mse_loss(y_hat, y):
    """ computes mean squared error loss """
    return ((y - y_hat)**2).mean()

Nu skal vi teste det. Vi har allerede outputtet og de ønskede (reference) værdier for test-sættet, så vi kan beregne tabværdien:

In [ ]:
loss_test = mse_loss(y_hat, y)
loss_test

Den mindre tabe, jo bedre.

I virkeligheden behøver du ikke at beregne tabet manuelt, som vi gjorde i kodeblokken ovenfor. PyTorch har allerede implementeret mange tabfunktioner, der er specielt designet til at blive brugt i træningsprocessen (f.eks. til beregning af gradienter). Her er en for MSE:

In [ ]:
loss_function = nn.MSELoss()
loss_test = loss_function(y_hat, y)

loss_test

Som du kan se, giver den værdi, der er identisk med det, vi fik ved brug af vores egen manuelle implementering af MSE-tabstab.

Som vi nævnte ovenfor, er der mange forskellige måder at måle tabet på, så MSE er ikke den eneste, der bruges. For eksempel bruges i tilfælde af binær klassifikation en anden funktion, *Binary Correlation Entropy* (BCE). I tilfælde af klassifikation med flere klasser kan man bruge *Cross Entropy Loss*.

Du behøver ikke at kende dem alle, husk bare at tabfunktionen viser dig (og din model), hvor stor forskellen er mellem den ønskede output og dem, din model beregner nu.

Derudover fortæller det modellen, hvordan man beregner gradienter - en række trin, som lader ANN ændre vægtene for at gøre tabet mindre. Denne proces med at reducere tabet ved gradvist at opdatere vægtene kaldes *[gradient descent](https://www.ibm.com/topics/gradient-descent)*, og det er den primære måde at træne enhver ANN model på. Nu kan vi diskutere træningen i detaljer.

### Øvelse 1

Udfyld følgende to tabeller (du kan gøre det f.eks. i Excel eller manuelt på papir), beregn MSE for hver og kommenter, hvor godt MSE beskriver klassificeringsresultaterne:


*Case 1*

| $y$ | $\hat{y}$ | $(y - \hat{y})$ | $(y - \hat{y})^2$ |
| --:| ---------:| ---------------:| -----------------:|
| -1 | 0.2 | - | - |
| -1 | 0.4 | - | - |
|  1 | -0.2 | - | - |
|  1 | -0.4 | - | - |

*Case 2*

| $y$ | $\hat{y}$ | $(y - \hat{y})$ | $(y - \hat{y})^2$ |
| --:| ---------:| ---------------:| -----------------:|
|  1 | 0.2 | - | - |
|  1 | 0.4 | - | - |
| -1 | -0.2 | - | - |
| -1 | -0.4 | - | - |


## Gradient og optimering

Tabelfunktionen bruges ikke kun til at vurdere kvaliteten af de forudsagte værdier, men også til at beregne gradienter for vægtene — hvordan vægtene skal ændres for at opnå en mindre tab på næste iteration (forbedre modellen).

Gradienterne er tilføjelser til vægtene, $\Delta w_1, \Delta w_2, \Delta w_3, \Delta w_4$ og til bias, $\Delta b$, som bruges til at beregne nye vægte for modellen. Den enkleste måde at beregne de nye vægte er følgende:

$w_1 = w_1 - \alpha \Delta w_1$<br>
$w_2 = w_2 - \alpha \Delta w_2$<br>
$w_3 = w_3 - \alpha \Delta w_3$<br>
$w_4 = w_4 - \alpha \Delta w_4$<br>
$bias = bias - \alpha \Delta b$

Som du kan se ovenfor, bruges tilføjelserne ikke direkte, men der er en yderligere parameter $\alpha$, som skal være mellem 0 og 1. For eksempel, hvis $\alpha = 0.01$, vil ændringen i vægtene være 1% af den beregnede tilføjelse.

Parametret $\alpha$ kaldes en *læringshastighed* og er nødvendig for at bremse læringsprocessen og gøre den mere jævn. Du vil se nogle eksempler senere i denne klasse.

Gradienterne beregnes af tabelfunktionsobjektet (i eksemplet ovenfor er det `nn.MSELoss()`), mens opdateringen af vægtene udføres af en anden funktion — en *optimerer*. Der er adskillige optimerer tilgængelige, hvoraf en af følgende to som regel er et godt valg:

* *GD* eller *SGD* (stochastic gradient descent) — den enkleste optimerer, som fungerer præcis som vist i ligningerne ovenfor. Simpel og ligetil. Navnet *gradient descent* betyder, at du beregner gradienten for at finde trin for vægtene, som får tabet til at gå nedad (nedstigning).
* *Adam* er en mere sofistikeret optimerer, som opdaterer vægtene på en lidt mere kompleks måde end vist ovenfor. Den har flere parametre at finjustere og er normalt mere effektiv.

I alle eksempler vil vi bruge *SGD*, da det er mere enkelt, men du kan prøve *Adam* senere.

## Træning af ANN model

Ideen med træning af ANN er som følger:

1. Anvend ANN til at beregne output baseret på inputs fra træningssættet.
2. Beregn tab baseret på det nuværende output og de ønskede outputværdier.
3. Beregn gradienter og opdater vægte, således at tabet næste gang er mindre.

Det første skridt kaldes *forward propagation* fordi data-værdier strømmer fra venstre (inputs) til højre (outputs) gennem alle neuroner imellem (vi har kun én neuron indtil videre i vores model, men det betyder ikke noget — hvad der virker for en vil virke for tusindvis).

Det tredje skridt kaldes *back propagation*, fordi gradienterne beregnes for output-neuronen først. Derefter for neuronerne tilbage mod outputtet, og så videre, indtil alle neuroner får opdateret de nye vægte baseret på gradienterne.

Disse tre trin gentages, indtil et kriterium er opfyldt, f.eks. at tabet ikke bliver mindre længere. Hver gang en model udfører alle tre trin for alle rækker i træningssættet, tager det en *epoke*. Så hvis du kører træningsprocessen i 10 epoker, betyder det, at disse tre trin gentages 10 gange for alle rækker i træningssættet.

Lad os implementere dette for vores model. Først og fremmest lad os forberede X- og y-værdier fra vores træningssæt (så de er numeriske og konverteret til Torch tensor)

In [ ]:
# select X variables, convert them to torch tensor and make them to have type float
X_train_values = d_train.iloc[:, 1:5].values
X_train = torch.tensor(X_train_values).float()

# compare species values with target class to get logical target values
c_train = d_train["Species"]

# convert logical values to numeric, so it is +1 if flower is virginica and -1 otherwise
y_train_dummy = (c_train  == "virginica") * 2.0 - 1.0

# convert the numeric values to torch tensor, make them float and transpose to a column
y_train = torch.tensor([y_train_dummy.values]).float().t()

Nu skal vi definere antallet af epochs til træning og tabfunktionen:

In [ ]:
# number of epochs to train the model
nepochs = 20


# define a loss function
loss_function = nn.MSELoss()

Hvor mange epoker der skal bruges, afhænger af sagen, normalt kan 100 epoker bruges som et startpunkt. Vi vil bruge 20 bare for at gøre outputtet kortere.

Vi er klar til at træne modellen.

Før vi starter, vil vi fastsætte tilstanden af tilfældighedsgenerator (som bruges til at initialisere vægtene). Dette er nødvendigt for at få reproducerbare resultater nedenfor, så når vi genkører koden, vil resultaterne være de samme. Hvis du senere vil prøve denne kode med virkelig tilfældige vægte, skal du bare kommentere disse to linjer.

In [ ]:

# fix the state of random numbers generator
seed = 11
#torch.manual_seed(seed)

# initialize a new model
model = SimpleModel()

# define optimizer which will compute gradients — do the learning, back propagation.
#
# parameter "lr" is learning rate it tells how large changes
# the weights will have (so it regulates how fast the learning process is)
# - if "lr" is too small your model can stuck and never reach the optimal model
# - if "lr" is too large your model can overshoot the optimal model
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# set model to training mode
model.train()

# training loop
for epoch in range(nepochs):

    # the gradients to zero
    optimizer.zero_grad()

    # 1. forward pass
    y_hat = model(X_train)

    # 2. compute the loss
    loss = loss_function(y_hat, y_train)

    # 3. backward pass and optimize weights
    loss.backward()
    optimizer.step()

    # show how big the loss is
    print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

Hvis du kommenterer linjen, der fryser tilfældigt nummergenerator og klikker på knappen "kør" flere gange, vil du se, at tabværdierne er forskellige hver gang du geninitialiserer og træner din model. Det skyldes, at de indledende vægte er indstillet til tilfældige numre, så træningsresultatet ikke er forudbestemt.

Derfor er det vigtigt at fryse tilstanden af hensyn til læring. Fjern venligst kommentaren og sørg for, at resultaterne er reproducerbare.

Lad os se, hvordan det klarer sig på træningssættet.

In [ ]:
# set model to predictions mode
model.eval()

# apply model to training set
y_hat = model.forward(X_train)

# convert output to numpy array
y_hat_arr = y_hat.t().detach().numpy()

# combine with reference values
res = pd.DataFrame({
    "Reference": d_train["Species"],
    "Predicted": y_hat_arr[0]
})

# compute statistics
TP = sum((res["Reference"] == "virginica") & (res["Predicted"] > 0))
TN = sum((res["Reference"] != "virginica") & (res["Predicted"] < 0))
FP = sum((res["Reference"] != "virginica") & (res["Predicted"] > 0))
FN = sum((res["Reference"] == "virginica") & (res["Predicted"] < 0))

sens = TP / (TP + FN)
spec = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)

(sens, spec, acc)


Ikke dårligt for selvtræning på 30 epoker. Og da dette er en meget simpel en-neuronsmodel, kan du få og se på vægtene og bias for denne neuron:

In [ ]:
# show weights and bias of the trained model
(model.layer1.weight, model.layer1.bias)

Som du kan se, er den vigtigste input i dette tilfælde *Blomstlængde*, da den har den største vægt på $0.56$. Den mindst vigtige er den anden (*Sepalbredde*), som har en vægt på $-0.03$. Den første input (*Sepallængde*) bidrager også negativt i denne kombination, ligesom den sidste.

Med andre ord implementerer vores kunstige neurale netværk følgende lineære model:

$\hat{Y} = -0.36 \times X_1 - 0.03 \times X_2 + 0.56 \times X_3 - 0.27 \times X_4 + 0.07$

Nu kan vi anvende vores model på testsættet.

In [ ]:
# prepare X and y values for the training set
X_test = torch.tensor(d_test.iloc[:, 1:5].values).float()

c_test = d_test["Species"]
y_test = (c_test == "virginica") * 2.0 - 1.0
y_test = torch.tensor([y_test.values]).float().t()

# apply model to test set
y_hat = model.forward(X_test)

# convert output to numpy array
y_hat_arr = y_hat.t().detach().numpy()

# combine with reference values
res = pd.DataFrame({
    "Reference": c_test,
    "Predicted": y_hat_arr[0]
})

# compute statistics
TP = sum((res["Reference"] == "virginica") & (res["Predicted"] > 0))
TN = sum((res["Reference"] != "virginica") & (res["Predicted"] < 0))
FP = sum((res["Reference"] != "virginica") & (res["Predicted"] > 0))
FN = sum((res["Reference"] == "virginica") & (res["Predicted"] < 0))

sens = TP / (TP + FN)
spec = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)

(sens, spec, acc)

For testsættet fungerer det endnu bedre (måske fordi testsættet er mindre).

### Øvelse 2

Nu leg lidt med denne kode. Forsøg at fjerne `torch.manual_seed()` instruktionen og kør træningen flere gange. Prøv at øge antallet af epoker og se, hvordan det påvirker kvaliteten af klassificeringen både for træningssættet og testsættet. Prøv at ændre læringshastigheden (gør den mindre eller større). Få den bedst mulige model og rapporter resultaterne.

## Aktiveringsfunktion og neuronlag

Hvordan gør man modellen endnu mere effektiv? Det mest oplagte er at øge antallet af lag og forbinde dem alle sammen. Dog vil det samlede antal vægte i dette tilfælde multipliceres, og du kan ende op med en model med tusindvis af vægte at finjustere. I dette tilfælde har du brug for flere objekter for at kunne træne, validere og teste det ordentligt.

Den anden måde at gøre ANN-modellen mere effektiv er at gøre den ikke-lineær. Den enkleste måde at introducere ikke-linearitet på er at supplere hver lineær neuron med hvad der kaldes en *aktiveringsfunktion*.

Aktiveringsfunktionen ændrer outputtet afhængigt af dets værdi. Den enkleste aktiveringsfunktion kaldes **Rectified Linear Unit (ReLU)**. Den fungerer som følger: Hvis outputtet er negativt, gør den det til nul. Men når outputtet er positivt, beholder den det bare som det er.

På billedet nedenfor kan du se to forskellige aktiveringsfunktioner, den blå er ReLU:

<img src="./illustrations/ReLU_and_GELU.svg" style="width:300px">

Lad os implementere ANN med 3 lag. Første lag vil bestå af 8 neuroner. Hver neuron vil have 4 inputs og 1 output, så dette lag vil have 8 outputs. Andet lag vil bestå af 4 neuroner, hver neuron har 8 inputs og 1 output, så dette lag vil have fire outputs. Endelig vil det sidste lag være det samme som vi har i vores nuværende model — én neuron med 4 inputs og 1 output.

>**Note til læreren**<br>tegn arkitekturen på en sort tavle.

Neuroner i de to første lag vil også have ReLU aktiveringsfunktion for outputtet.

Her er implementeringen:

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class NewModel(nn.Module):
    """ class for three layers ANN """

    def __init__(self):
        super(NewModel, self).__init__()
        self.layer1 = nn.Linear(4, 8)  # first layer: 4 inputs and 8 outputs
        self.layer2 = nn.Linear(8, 4)  # second layer: 8 inputs and 4 outputs
        self.layer3 = nn.Linear(4, 1)  # first layer: 4 inputs and 1 output

    def forward(self, x):
        x = F.relu(self.layer1(x)) # pass inputs through the first layer and apply ReLU activation
        x = F.relu(self.layer2(x)) # pass output from layer 1 through the second layer + ReLU activation
        y_hat = self.layer3(x) # pass output from layer 2 through the third layer (no relu)
        return y_hat

Og her er koden, der implementerer resten (træning og test).

Som du kan se i dette tilfælde beregner vi tab for træningssættet og for testsættet for hvert epoch. Så vi kan opdage situationen, hvor modellen bliver dårligere for testsættet, og stoppe. Denne proces kaldes *validering* og det lader os undgå overtræning af modellen, når den fungerer perfekt for træningssættet og dårligt for testsættet. Denne situation kaldes også *overfitting*.

In [ ]:
# we use another manual seed here to get reproducible outcome
torch.manual_seed(42)

# number of epochs to train the model
nepochs = 300

# define a loss function
loss_function = nn.MSELoss()

# initialize the new model
model = NewModel()

# define optimizer which will compute gradients — do the learning.
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# training loop
for epoch in range(nepochs):  # Number of training epochs

    # train
    model.train()
    optimizer.zero_grad()
    y_hat_train = model(X_train)
    train_loss = loss_function(y_hat_train, y_train)
    train_loss.backward()
    optimizer.step()

    # validate
    model.eval()
    y_hat_test = model(X_test)
    test_loss = loss_function(y_hat_test, y_test)

    # show how big the loss is
    print(f'Epoch {epoch}, train loss: {train_loss.item():.4f} - test loss {test_loss.item():.4f}')

Som du kan se, bruger vi flere epoker i dette tilfælde, da modellen er lidt mere kompleks.

Lad os se, hvordan den klarer sig på testsættet:

In [ ]:
# set model to prediction mode
model.eval()

# apply model to test set
y_hat = model.forward(X_test)

# convert output to numpy array
y_hat_arr = y_hat.t().detach().numpy()

# combine with reference values
res = pd.DataFrame({
    "Reference": c_test,
    "Predicted": y_hat_arr[0]
})

# compute statistics
TP = sum((res["Reference"] == "virginica") & (res["Predicted"] > 0))
TN = sum((res["Reference"] != "virginica") & (res["Predicted"] < 0))
FP = sum((res["Reference"] != "virginica") & (res["Predicted"] > 0))
FN = sum((res["Reference"] == "virginica") & (res["Predicted"] < 0))

sens = TP / (TP + FN)
spec = TN / (TN + FP)
acc = (TP + TN) / (TP + TN + FP + FN)

(sens, spec, acc)

Som vi nævnte ovenfor, strengt taget skal vi til dette sidste trin bruge et andet sæt prøver, uafhængigt af det der blev brugt til træning og validering. Vi bryder denne regel her af illustrative formål, kun fordi vores datasæt er lille.

Vi kan visualisere de forudsagte værdier for bedre forståelse:

In [ ]:

import matplotlib.pyplot as plt
plt.scatter(res["Reference"], res["Predicted"])
plt.plot(plt.xlim(), [0, 0], color = "black", linestyle="--")
plt.ylabel("Predicted output")

Det skal bemærkes, at hverken netværket vi bruger i det sidste eksempel, eller den valgte tabssfunktion er specifikt god til klassificering. Men selv med denne valg ser resultatet godt ud. Lad os lære, hvordan vi kan forbedre dette.

## Flereklassifikation

En af de mest almindelige måder at lave en klassifikationsmodel med en kunstig neuralt netværk (ANN) på, er ved at bruge outputlaget med flere neuroner - ét for hver klasse. Så, for binær klassifikation får vi to outputs, for klassifikation blandt tre klasser - tre og så videre. Men hvordan træffes klassifikationsbeslutningen i dette tilfælde, og hvilken tabfunktion skal bruges?

Idéen er lignende afstemning. Beslutningen træffes ved at vælge output med den største værdi. For eksempel, hvis de forudsagte værdier i outputlaget er [0.23, 0.89, -0.01], så "vinder" det andet output, og den forudsagte labels indeks vil være 1 (husk, i Python starter indekser fra 0, så vi har 0, 1 og 2 i stedet for 1, 2 og 3). Dette betyder følgende:

1. Vi skal have lige så mange outputs i det sidste (output)lag som antallet af klasser, vi har.
2. Vi skal oprette en vektor med referenceklassernes indeks.
3. Vi skal bruge en speciel tabfunktion, der fungerer bedst i dette tilfælde.

Lad os implementere dette ved at oprette en model, der vil forudsige klasselabelen for Iris-dataene. Denne gang nogen af de tre labels. Lad os antage, at labelen `"setosa"` vil have indeks 0, `"versicolor"` vil have indeks 1, og `"virginica"` vil have indeks 2.

Lad os oprette en ordbog med indeksene og labelerne:

In [ ]:
# create vector with classes, so we can get a label by its index
classes = ["setosa", "versicolor", "virginica"]

# create dictionary so we can get index by label
class_to_idx = {"setosa": 0, "versicolor": 1, "virginica": 2}

Nu skal vi forberede datasættene. Vi har allerede gjort dette ovenfor, men lad os gøre det igen, og denne gang lad os også oprette en vektor med referenceklasseindeks.

In [ ]:
# create tensor with predictor values
X_values = d.iloc[:, 1:5].values
X = torch.tensor(X_values).float()

# create tensor with class indices
label_values = [class_to_idx[label] for label in d["Species"]]
labels = torch.tensor(label_values).long()

# show it on the screen to check
labels

Som du husker gør tilføjelse af `float()` til tensor værdierne til flydende punkt numre. Tilføjelse af `long()` gør dem til heltalsværdier. At have label indekser som heltal er påkrævet af tabfunktionen, vi kommer til at bruge.

Nu vil vi splitte begge tensors op i trænings- og testsæt ligesom vi gjorde tidligere:

In [ ]:
# generate indices of rows for training and test set
train_ind = d["Id"] % 5 != 0
test_ind = d["Id"] % 5 == 0

# select rows and label indices for the training set
X_train = X[train_ind, :]
labels_train = labels[train_ind]

# select rows and label indices for the test set
X_test = X[test_ind, :]
labels_test = labels[test_ind]

# show test set labels
labels_test

Som du kan se, denne gang i stedet for at lave delmængder for data rammen, vi oprettede tensors først og derefter delmængde tensoerne. Denne måde er lidt kortere og måske mere klar.

Nu vil vi oprette en ny klasse med ANN model med tre outputs:

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class MultiClassModel(nn.Module):
    """ class for three layers ANN """

    def __init__(self):
        super(MultiClassModel, self).__init__()
        self.layer1 = nn.Linear(4, 8)  # first layer: 4 inputs and 8 outputs
        self.layer2 = nn.Linear(8, 16)  # first layer: 8 inputs and 16 outputs
        self.layer3 = nn.Linear(16, 3)  # first layer: 16 inputs and 3 outputs

    def forward(self, x):
        x = F.relu(self.layer1(x)) # pass inputs through the first layer and apply ReLU activation
        x = F.relu(self.layer2(x)) # pass output from layer 1 through the second layer + ReLU activation
        y_hat = self.layer3(x) # pass output from layer 2 through the third layer (no relu)
        return y_hat

Som du kan se, modellen ligner meget det, vi havde før, vi har blot ændret antallet af input/output.

> **Bemærkning til læreren:**<br>tegn modellen skematisk på en sort tavle.

Nu skal vi definere tabfunktionen. I dette særlige tilfælde, når beslutningen træffes ved afstemning, er den bedst egnede funktion krydsentropitab:

In [ ]:
# define a loss function
loss_function = nn.CrossEntropyLoss()

Alt er klar, lad os initialisere og træne modellen (ligesom i eksemplerne ovenfor vil vi fryse tilfældighedsgeneratorens tilstand for at opnå reproducerbare resultater):

In [ ]:
# seed the random number generator to get reproducible outcome
torch.manual_seed(12)

# initialize the new model
model = MultiClassModel()

# define optimizer which will compute gradients — do the learning.
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# number of epochs to train the model
nepochs = 100

for epoch in range(nepochs):  # Number of training epochs

    # train
    model.train()
    optimizer.zero_grad()
    labels_predicted = model(X_train)

    loss = loss_function(labels_predicted, labels_train)

    loss.backward()
    optimizer.step()
    train_loss = loss.item()

    # validate
    model.eval()
    labels_predicted = model(X_test)
    loss = loss_function(labels_predicted, labels_test)
    val_loss = loss.item()

    # show how big the loss is
    print(f'Epoch {epoch}, train loss: {train_loss:.4f} - validation loss {val_loss:.4f}')


Indtil videre går det godt, selvom som du kan se, fortsætter den validerende taben med at mindske, så måske er 100 epoker ikke nok. Vi vil vende tilbage til dette senere. Lad os lære, hvordan man laver forudsigelser.

Lad os lave forudsigelser for begge sæt og se på de forudsagte værdier for testsættet først:

In [ ]:
# set model to prediction mode
model.eval()

# apply model to train and test sets
output_train = model.forward(X_train)
output_test = model.forward(X_test)

# show predictions for the test set
output_test

Hvis du kigger nøje efter, kan du finde ud af, at for de første ti rækker (hvor vi har *setosa* prøver) er den største af de tre værdier faktisk den første. For de to andre er det ikke så tydeligt. Lad os anvende `max` funktionen på hver række og spørge ikke efter værdien, men efter positionen (indeks) hvor den største værdi er placeret:

In [ ]:
# get indices of largest values for each row of computed outputs
_, predictions_train = torch.max(output_train, 1)
_, predictions_test = torch.max(output_test, 1)

predictions_test

Ja, den forudsagte klasselabel-indeks for setosa er god, men de andre er ikke helt perfekte endnu. Vi vil fikse det senere.

Fordi vi her har tre klasser, vil det være for tidskrævende at beregne klassifikationsstatistikker for hver enkelt. Lad os lære en ny måde at vurdere klassificeringsresultaterne på — via en contingency-tabel eller [forvirringsmatrix](https://da.wikipedia.org/wiki/Forvirringsmatrix). Den viser simpelthen alle mulige kombinationer af reference- og forudsagte klasselabels:

In [ ]:
import numpy as np

# compute contingency table for training and test set
ct_train = np.zeros((3, 3))
ct_test = np.zeros((3, 3))

for i in range(3):
    for j in range(3):
        ct_train[i, j] = sum((labels_train == i) & (predictions_train == j))
        ct_test[i, j] = sum((labels_test == i) & (predictions_test == j))

(ct_train, ct_test)


Tabellen for den ideelle klassifikation skal have nul for alle værdier undtagen på diagonalen (hvor forudset og reference mærkning matcher). Som du kan se, har vi en perfekt match for den første (0, "setosa") og den anden (2, "versicolor") klasse, men "virginica" klassen er ikke forudsagt godt. Seks blomster af denne klasse blev forkert forudsagt som *versicolor* og fire blev korrekt forudsagt som *virginica*.

Du kan også beregne denne tabel for relative værdier (procent), hvilket er nemmere at bruge, når antallet af individer i forskellige klasser ikke er det samme:

In [ ]:
# compute contingency table for training and test set with relative values
ct_train = np.zeros((3, 3))
ct_test = np.zeros((3, 3))
for i in range(3):
    n_train = sum(labels_train == i)
    n_test = sum(labels_test == i)
    for j in range(3):
        ct_train[i, j] = sum((labels_train == i) & (predictions_train == j)) / n_train
        ct_test[i, j] = sum((labels_test == i) & (predictions_test == j)) / n_test

(ct_train, ct_test)

Lad os lære at visualisere dette ved at lave en varmekort.

In [ ]:
# heatmap plot for contrast table
plt.imshow(ct_test, clim = [0, 1])
plt.colorbar()

Nu skal vi tilføje klasselabels og vise tallene for hver celle i tabellen:

In [ ]:
# heatmap plot for contrast table
plt.imshow(ct_test, clim = [0, 1])
plt.colorbar()
plt.gca().set_xticks(range(3), classes)
plt.gca().set_yticks(range(3), classes)
for i in range(3):
    for j in range(3):
        plt.text(i, j, round(ct_test[j, i], 3), color = "white" if ct_test[j, i] < 0.5 else "black")

Lad os oprette flere funktioner for at kunne genbruge koden, vi har skrevet ovenfor.

In [ ]:
def train(model, X_train, labels_train, X_val, labels_val, nepochs = 100, lr = 0.01):
    """ trains any classification model using provided data, number of epochs and learning rate """

    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for epoch in range(nepochs):
        model.train()
        optimizer.zero_grad()
        labels_predicted = model(X_train)
        loss = loss_function(labels_predicted, labels_train)
        loss.backward()
        optimizer.step()
        train_loss = loss.item()

        model.eval()
        labels_predicted = model(X_val)
        loss = loss_function(labels_predicted, labels_val)
        val_loss = loss.item()

        print(f'Epoch {epoch}, train loss: {train_loss:.4f} - validation loss {val_loss:.4f}')

In [ ]:
def predict(model, X):
    """ get ANN model and tensor with predictors and returns predicted class label indices """
    model.eval()
    output = model.forward(X)
    _, labels = torch.max(output, 1)
    return labels

In [ ]:
def table(reference, predicted):
    """ computes contingency table for predicted and reference class label indices """
    indices = reference.unique()
    n = len(indices)
    ct = np.zeros((n, n))
    for i in range(n):
        ni = sum(reference == indices[i])
        for j in range(n):
            ct[i, j] = sum((reference == indices[i]) & (predicted == indices[j])) / ni

    return ct

In [ ]:
def ct_heatmap(ct, classes):
    """ shows heatmap for the contingency table """
    plt.imshow(ct, clim = [0, 1])
    plt.colorbar()

    n = len(classes)
    plt.gca().set_xticks(range(n), classes)
    plt.gca().set_yticks(range(n), classes)
    for i in range(n):
        for j in range(n):
            plt.text(i, j, round(ct[j, i], 3), color = "white" if ct[j, i] < 0.5 else "black")

Som du kan se, returnerer funktionen 'train()' ingenting. Det er fordi 'model', som vi passerer til denne funktion som det første argument, er et objekt, der har sine egne metoder, som opdaterer dette objekt internt. Så selvom al træning sker inde i denne funktion, får objektet udenfor alle nødvendige opdateringer.

Desuden kan vi også bruge denne funktion til enhver anden model.

 Lad os se, hvordan de nye funktioner virker. Lad os træne og initialisere modellen igen og træne den igen med flere epoker og en mindre læringsrate:

In [ ]:
torch.manual_seed(12)
model = MultiClassModel()
train(model, X_train, labels_train, X_test, labels_test, nepochs = 1000, lr = 0.01)

Og tjek hvordan det klarer sig:

In [ ]:
predictions_train = predict(model, X_train)
predictions_test = predict(model, X_test)

ct_train = table(labels_train, predictions_train)
ct_test = table(labels_test, predictions_test)

plt.figure(figsize = (10, 5))

plt.subplot(1, 2, 1)
ct_heatmap(ct_train, classes)
plt.title("Train")

plt.subplot(1, 2, 2)
ct_heatmap(ct_test, classes)
plt.title("Test")

Med 1000 epoker og en mindre læringsrate fungerer den nye arkitektur næsten perfekt!

### Øvelse 3

I filen `IrisHeatmap.csv` finder du nye datapunkter, som indeholder nye målinger, men ikke har en kolonne med art eller en kolonne med ID'er (så den har kun fire kolonner med målinger). Indlæs dataene fra filen, og anvend derefter det ANN, du lige har trænet, for at få forudsigelserne.

Efterfølgende lav et spredningsplot, hvor x- og y-værdierne er Blomsterbladslængde og Blomsterbladsbredde, som du har fået fra filen. Vis punkter, forudsagt som *setosa*, ved hjælp af rød farve, punkter forudsagt som *versicolor* som grønne, og punkter forudsagt som *virginica* som blå. Kommenter plottet. Hvad viser det faktisk?

Prøv at tilføje forudsigelser for trænings- og testsættet til plottet. I dette tilfælde bliver du nødt til at gøre forudsigelserne for de nye data vist semitransparente. Brug parameteren `alpha = 0.15` i `plt.scatter()`-funktionen til dette.

## Visualisering af træningsprocessen

Det kan være nyttigt at se, hvordan trænings- og valideringstab ændrer sig med epokerne. Lad os modificere metoden `train_model()` for at indsamle denne information og returnere den til brugeren:

In [ ]:
def train(model, X_train, labels_train, X_test, labels_test, nepochs = 100, lr = 0.01):
    """ trains any classification model using provided data, number of epochs and learning rate """

    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    train_losses = np.zeros(nepochs)
    val_losses = np.zeros(nepochs)

    for epoch in range(nepochs):
        model.train()
        optimizer.zero_grad()
        labels_predicted = model(X_train)
        loss = loss_function(labels_predicted, labels_train)
        loss.backward()
        optimizer.step()

        train_losses[epoch] = loss.item()

        model.eval()
        labels_predicted = model(X_test)
        loss = loss_function(labels_predicted, labels_test)
        val_losses[epoch] = loss.item()

        print(f'Epoch {epoch}, train loss: {train_losses[epoch]:.4f} - validation loss {val_losses[epoch]:.4f}')

    return (train_losses, val_losses)

Nu skal vi træne modellen igen, denne gang ved hjælp af 5000 epoker, få tabingsværdierne og lave en plot.

In [ ]:
# seed the random number generator to get reproducible outcome
torch.manual_seed(12)

# re-initialize and train the model
model = MultiClassModel()
train_losses, val_losses = train(model, X_train, labels_train, X_test, labels_test,
                                 nepochs = 5000, lr = 0.01)

In [ ]:
# show plot with both losses
plt.plot(train_losses, label = "train")
plt.plot(val_losses, label = "val")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.ylim([0, 0.2])

Som du kan se, ændrer valideringen næsten ikke tab efter cirka 3000 epoker. Så der er ingen grund til at køre modellen så langt.

Lad os geninitialisere modellen (dette er nødvendigt for at sætte alle startvægte til tilfældige tal, ellers vil vægte til allerede trænet model blive brugt som startpunkt) og træne den igen, men denne gang vil vi bruge en stor læringsrate:

In [ ]:
torch.manual_seed(12)
model = MultiClassModel()
train_losses, val_losses = train(model, X_train, labels_train, X_test, labels_test,
                                 nepochs = 5000, lr = 0.2)

In [ ]:
plt.plot(train_losses, label = "train")
plt.plot(val_losses, label = "val")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")

Du kan se nogle mærkelige mønstre, "toppe", på plottet. Disse mønstre er typiske for en stor læringsrate, hvor optimeringsalgoritmen ændrer vægtene på neuronerne for meget, så modellen "springer" frem og tilbage.

Dette kan sammenlignes med at køre en bil. Hvis du accelererer og bremser langsomt, overvåger situationen på vejen og handler i god tid (proaktivt), vil din bil bevæge sig jævnt og behageligt for dig og dine passagerer. Men situationen vi ser på plottet ovenfor svarer til tilfældet hvor du accelererer og bremser konstant.

Dette plot hjælper med at finde den optimale læringsrate og antal epoker.

## Gem og indlæs modeltilstand

Endelig lad os lære, hvordan man gemmer og indlæser modeltilstanden - alle interne parametre, som inkluderer vægte og forudindstillinger af alle lag osv. Dette kan være nyttigt i følgende situationer:

1. Du kan gemme modeltilstanden i en separat variabel efter hvert epoch, men kun hvis valideringstab bliver bedre. Dette hjælper med at undgå overfitting, når du træner din model for længe, og den klarer sig dårligere på valideringssættet end f.eks. 50 epoch'er før. Vi vil overveje dette tilfælde i den næste klasse.

2. Gem modellen til en fil til senere brug. For eksempel kan du sende den til dine klassekammerater eller kolleger. Eller bare gem den for at fortsætte træningsprocessen i morgen. Eller for at genbruge den til forudsigelser. Mange muligheder.

Lad os først se, hvordan man får modellens tilstand. Prøv at køre den næste kode først:

In [ ]:
model.parameters()

In [ ]:
for parameter in model.parameters():
    print(parameter.shape)

Du kan se en samling af torch tensors (ligesom NumPy-arrays, som vi husker). For eksempel har den første tensor 8 rækker og 4 kolonner. Kan du gætte hvorfor?

Fordi i den første lag af din model har du 8 neuroner. Hvert neuron har 4 inputs og 1 output. Og output beregnes ved at tage en vægtet sum af inputs plus bias. Så for hvert neuron har du brug for 5 parametre — 4 vægte og 1 bias.

Den første tensor af størrelse 8x4 indeholder vægten for hver af de 8 neuroner, og den anden tensor (vektor med 8 værdier) indeholder biases.

Det andet lag har 16 neuroner, hvert neuron har 8 inputs og 1 output. Så hvert neuron har 9 parametre — 8 vægte og 1 bias. Det er præcis hvad du har i det næste par af tensors, den ene af størrelse 16x8 indeholder vægte og den ene med 16 værdier indeholder biases.

Endelig har den sidste lag 3 neuroner med 16 inputs og 1 output hver, så de sidste to tensors indeholder vægte og biases for disse neuroner.

Faktisk kan du se strukturen af din model og antallet af parametre ved at bruge metoden `summary()` fra biblioteket `torchinfo`:

In [ ]:
from torchinfo import summary
summary(model)

Du kan også beregne det samlede antal parametre i din model manuelt ved simpelthen at tage summen af alle elementer i hver tensor:

In [ ]:
npar = 0
for parameter in model.parameters():
    npar = npar + parameter.numel()

print(f"Total number of parameters in this ANN: {npar}")

Du kan selvfølgelig se alle vægtene (outputtet vil dog være langt):

In [ ]:
for parameter in model.parameters():
    print("------")
    print(parameter)

Strengt taget kan du tage alle disse værdier, kopiere dem til Excel og køre forudsigelser i Excel (du skal bare huske at anvende ReLU-funktionen på hver output). Skør idé, men det kan lade sig gøre. Dette viser, at ANN ikke er en raketvidenskab, men en meget simpel, ligetil og kraftfuld metode.

Der er en anden måde at få modelparametrene med al nødvendig yderligere information på — statens ordbog. Der er en metode til det:

In [ ]:
model.state_dict()

Som du kan se, er outputtet lignende med det, vi har set før, men denne gang er det organiseret som en ordbog, så Torch vil kende et niveaunavn og parameternavn.

Vi kan gemme tilstanden i en variabel ved at tage en dyb kopi:

In [ ]:
# we need to load a special function which creates copy of complex objects
from copy import deepcopy

# save parameters of current model to a variable
model_state = deepcopy(model.state_dict())

Hvorfor har du brug for at tage en dyb kopi i stedet for bare at tildele status til en ny variabel? Fordi hvis du fortsætter med at træne din model, vil denne status også få alle opdateringer. Ved at tage en dyb kopi adskiller du på en måde den nuværende status fra de næste, hvilket gør den uafhængig.

Her er et eksempel på, hvordan du kan bruge det:

In [ ]:
# seed the random number generator to get reproducible outcome
torch.manual_seed(12)

# initialize a new model
new_model = MultiClassModel()

# make predictions — they will be very bad because the model is not trained
predictions = predict(new_model, X_test)
predictions

In [ ]:
# now lets load the parameters we saved from the trained model to this new model
new_model.load_state_dict(model_state)

# and make predictions
predictions = predict(new_model, X_test)
predictions

Og nu har vi perfekte forudsigelser uden at træne den nye model, men bare ved at genbruge parametrene fra den tidligere trænede model.

Hvis du vil gemme tilstanden til fil og sende den til nogen, eller indlæse den senere i en anden Python-script, kan du bruge to PyTorch-funktioner: `torch.save()` gemmer modeldictionary til en fil, og `torch.load()` indlæser den fra filen. Filen skal have filtypenavnet `.pth`.

 Lad os se, hvordan det virker:

In [ ]:
# save state dictionary to file
torch.save(model.state_dict(), "mymodel.pth")

Som du kan se, behøver vi ikke at tage en deepcopy i dette tilfælde, fordi det vil være i en separat fil. Hvis du kører det, får du filen 'mymodel.pth' inde i den nuværende mappe.

Der er ingen grund til at åbne den, da informationen indeni er kodet ved hjælp af binær format. Men du kan indlæse den og tildele den til modellen:

In [ ]:
# create a new model with random weights
another_model = MultiClassModel()

# load state from the file and assign it to the new model
model_state = torch.load("mymodel.pth")
another_model.load_state_dict(model_state)

# make predictions
predictions = predict(another_model, X_test)
predictions

Det virker!

### Øvelse

For at udføre denne øvelse skal du arbejde i par. En af medlemmerne skal træne et ANN-netværk på Iris data (lav en separat notebook og kopier al nødvendig kode derover). Træn det ved at bruge tilfældig initialisering og prøv forskellige hyperparametre (antal epoker, indlæringshastighed osv.). Når du har fået en god model, gem den i en fil og send den til din gruppekammerat via e-mail. Din gruppekammerats opgave er at indlæse modellen og anvende den på testsettet.

In [ ]:
# place your code here

## Leg med interaktiv ANN på web

Nu ved du alt, hvad du har brug for at vide til næste time. Vi anbefaler dog, at du leger lidt mere ved at bruge denne interaktive ANN-konstruktør, som du kan køre direkte i din webbrowser. Brug lidt tid på at lege med forskellige problemer, forskellige arkitekturer og hyperparametre:

[TensorFlow playground](http://playground.tensorflow.org)